# Data Wrangling on WeRateDogs



## Table of Contents
- [Introduction](#intro)
- [Gather](#gather)
- [Assess](#assess)
- [Clean](#clean)

<a id='intro'></a>
## Introduction

The main purpose of this project is to put into practice key concepts learnt.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import tweepy
%matplotlib inline

<a id='gather'></a>
## Gather    

Data gathering will be comprised of the following steps:   
**1.-** Import the tweeter archive for WeRateDogs into the workspace   
**2.-** Download image predictions data from a given URL   
**3.-** Gather data from the Tweeter API to collect more relevant information   

#### 1.- Import the tweeter archive
This file has already been provided as a manual download. As such, it's available locally and just need to import it using pandas read_csv method for reading flat files into a pandas dataframe

In [2]:
archive_df = pd.read_csv("twitter_archive_enhanced.csv")
archive_df.head(1)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None


#### 2.- Download image predictions data from a given URL   
Unlike the previous file, in this case the file URL was given. Hence, we download the file programatically as this is less prone to errors and eases reproducibility.    
First download the file using python library **_requests_** and save the information downloaded in a file. Then, import the file similarly to how it was done in step \#1 

In [3]:
url = "https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv"
image_predictions = requests.get(url)
file_name = url.split('/')[-1]
with open(file_name, mode='w', encoding =image_predictions.encoding) as f:
    f.write(image_predictions.text)
image_df = pd.read_csv(file_name,sep='\t')
image_df.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


#### 3.- Gather data from Tweeter by accessing their API 
The third step is to obtain various pieces of data from Tweeter directly by using their API. Please note that keys and tokens are stored in a separate file that is stored locally and is not part of source control. As such, these fields will need to be edited if this notebook was to used different tweeter app credentials

In [5]:
getData = lambda line: line.split(' ')[0]
with open('tweeterKeys.txt','r',encoding='utf-8') as f:
    consumer_key = getData(f.readline())
    consumer_secret = getData(f.readline())
    access_token = getData(f.readline())
    access_secret = getData(f.readline())

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth,wait_on_rate_limit=True,wait_on_rate_limit_notify=True)

with open('tweet_json.txt','w',encoding='utf-8') as f:
    with open('tweetIdsFailed.txt','w',encoding='utf-8') as ffail:
        for row in archive_df['tweet_id']:
            try:
                f.write(str(api.get_status(row)._json) + '\n')
            except tweepy.TweepError as e:
                ffail.write(str(row)+'\n')

Rate limit reached. Sleeping for: 480
Rate limit reached. Sleeping for: 556


<a id='assess'></a>
## Assess

#### Quality

#### Tidiness

<a id='clean'></a>
## Clean

### Missing Data

##### Define

##### Code

##### Test